In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate

# Mount to google drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'fastai-v3/covid19/data'

folder = 'ards'
path = Path(base_dir)
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

In [0]:
bs = 16
# bs = 16   # uncomment this line if you run out of memory even after clicking Kernel->Restart

In [0]:
path.ls()

In [0]:
classes = ['pneumocystis','streptococcus','nofinding','covid19','ards','sars']

In [0]:
for c in classes:
    print(c)
    verify_images(path/c, delete=True, max_size=500)

In [0]:
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4, bs=bs).normalize(imagenet_stats)

In [0]:
data.show_batch(rows=3, figsize=(7,6))

In [0]:
print(data.classes)
len(data.classes),data.c
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

## Training: resnet34

In [0]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [0]:
learn.fit_one_cycle(6)

In [0]:
learn.save('stage-1')

In [0]:
learn.unfreeze()

In [0]:
learn.lr_find()


In [0]:
# If the plot is not showing try to give a start and end learning rate
# learn.lr_find(start_lr=1e-5, end_lr=1e-1)
learn.recorder.plot()

In [0]:
learn.fit_one_cycle(10, max_lr=slice(1e-04,1e-03))

In [0]:
learn.save('stage-2')

In [0]:
learn.show_results()

## Results

In [0]:
learn.load('stage-2');

In [0]:
interp = ClassificationInterpretation.from_learner(learn)

In [0]:
interp.plot_confusion_matrix()

In [0]:
losses,idxs = interp.top_losses()
interp.plot_top_losses(15, figsize=(15,11))
# doc(interp.plot_top_losses)
# Show images in top_losses along with their prediction, actual, loss, and probability of actual class.
## interp.most_confused(min_val=2)

TESTING OUT MODEL

In [0]:
learn.export()

In [0]:
learn = load_learner(path)

In [0]:
img = open_image(path/'covid19'/'21DDEBFD-7F16-4E3E-8F90-CB1B8EE82828.jpeg')
img

In [0]:
pred_class,pred_idx,outputs = learn.predict(img)
pred_class